In [ ]:
import asyncio
import asyncpraw
import nest_asyncio
import pandas as pd
import random
import time

# Hàm khởi tạo, hàm extract commment, hàm check rate limit, hàm get all cmt của 1 post

In [31]:
from datetime import datetime


# 🔹 Hỗ trợ asyncio trong Jupyter Notebook
nest_asyncio.apply()

# 🔹 Số lượng request đồng thời tối đa
MAX_CONCURRENT_REQUESTS = 1

# 🔹 Khởi tạo Reddit API client
async def create_reddit_client():
    return asyncpraw.Reddit(
        client_id="pPEPZOQQcEgduy1bXHXaUQ",
        client_secret="v6Ta6emxFqC1hDWV6XWVPzwCNI_iQg",
        user_agent="SensitiveRaccoon1668",
        request_timeout=90
    )

# 🔹 Kiểm tra rate limit còn lại
async def check_rate_limit(reddit):
    try:
        await reddit.request("GET", "/api/v1/me")
        remaining = reddit.auth.limits.get("remaining", 60)  # Mặc định 60 nếu không có giá trị
        reset_time = reddit.auth.limits.get("reset_timestamp", time.time() + 60)
        print(f"Remaining requests: {remaining}, Reset time: {datetime.utcfromtimestamp(reset_time).strftime('%Y-%m-%d %H:%M:%S')}")
        return remaining, reset_time
    except Exception as e:
        print(f"⚠️ Không thể kiểm tra rate limit: {e}")
        return 60, time.time() + 60  # Mặc định nếu không lấy được dữ liệu

# 🔹 Hàm lấy toàn bộ comment của một bài post
async def get_all_comments_async(reddit, post_id, semaphore, max_retries=3):
    async with semaphore:  
        remaining, reset_time = await check_rate_limit(reddit)

        if remaining < 5:  # Nếu còn quá ít request, chờ đến reset
            wait_time = reset_time - time.time() + 1
            print(f"⏳ Rate limit sắp hết, chờ {wait_time:.2f}s...")
            await asyncio.sleep(wait_time)

        for attempt in range(max_retries):  # Thử tối đa max_retries lần
            try:
                print(f"📌 [Lần {attempt+1}] Đang lấy comment của post: {post_id}...")
                submission = await reddit.submission(id=post_id)
                await submission.comments.replace_more(limit=None)
                comments_list = submission.comments
                def extract_comments(comment_list):
                    all_comments = []
                    for comment in comment_list:
                        all_comments.append({
                            "post_id": post_id,
                            "comment_id": comment.id,
                            "comment_text": comment.body,
                            "comment_author": comment.author.name if comment.author else "Anonymous",
                            "comment_score": comment.score,
                            "comment_time": comment.created_utc
                        })
                        all_comments.extend(extract_comments(comment.replies))  
                    return all_comments

                comments = extract_comments(comments_list)
                print(f"✅ Hoàn tất post {post_id}, lấy được {len(comments)} comment.")

                # 🔹 Điều chỉnh tốc độ request dựa vào remaining
                delay = max(1, random.uniform(2, 6) / remaining)  
                print(f"⏳ Chờ {delay:.2f}s trước khi tiếp tục...")
                await asyncio.sleep(delay)

                return comments

            except Exception as e:
                print(f"⚠️ Lỗi khi lấy comment của post {post_id} (Lần {attempt+1}/{max_retries}): {e}")
                if attempt < max_retries - 1:
                    wait_time = 2 ** attempt  # Chờ lâu hơn sau mỗi lần thử
                    print(f"🔁 Thử lại sau {wait_time}s...")
                    await asyncio.sleep(wait_time)
                else:
                    print(f"❌ Bỏ qua post {post_id} sau {max_retries} lần thử.")
                    return []

# 🔹 Hàm xử lý nhiều post từ file CSV
async def process_posts_from_csv(csv_file):
    df = pd.read_csv(csv_file)
        # **Lọc chỉ lấy matchday từ 7 đến 12**
    df = df[(df["matchday"] >= 1) & (df["matchday"] <= 6)]
    post_ids = df["submission_id"].dropna().astype(str).tolist()
    print(f"🔄 Tổng số trận đấu cần cào: {len(post_ids)}\n")

    reddit = await create_reddit_client()
    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

    tasks = [get_all_comments_async(reddit, post_id, semaphore) for post_id in post_ids]
    results = await asyncio.gather(*tasks)

    await reddit.close()

    all_comments = [comment for post_comments in results for comment in post_comments]
    print(f"\n📊 Tổng số comment lấy được: {len(all_comments)}")
    return all_comments


In [ ]:

# 🔹 Chạy script
csv_file = r"D:\Desktop\tap tanh hoc code\.vscode\DAP\Reddit_Sentiment_Analysis\dataframe\full_post(finished_matches_order).csv"
comments = asyncio.run(process_posts_from_csv(csv_file))

# 🔹 Lưu kết quả vào file CSV
df_comments = pd.DataFrame(comments)


In [25]:
df_comments.to_csv("reddit_comments.csv", index=False)

print(f"\n✅ Hoàn thành! Đã lấy {len(comments)} comment và lưu vào reddit_comments.csv.")



✅ Hoàn thành! Đã lấy 35722 comment và lưu vào reddit_comments.csv.


In [40]:
# Load the CSV file into a DataFrame
df_comments = pd.read_csv(r'D:\Desktop\tap tanh hoc code\.vscode\DAP\reddit_comments_updated.csv')

# Group by 'post_id' and count the number of comments for each post
comment_counts = df_comments.groupby('post_id').size().reset_index(name='comment_count')

# Display the result
print(comment_counts)
total_comment = comment_counts['comment_count'].sum()
print(total_comment)

    post_id  comment_count
0   1etwv7n           3817
1   1eufbfh           2371
2   1eui1ym           1374
3   1eui26i            196
4   1eui3pz            268
5   1eui74j             44
6   1eullxa            601
7   1ev8sh0            755
8   1evbj42           4631
9   1ew9irk           1563
10  1f02vo3           2139
11  1f05t5x            829
12  1f05yy1             49
13  1f0627i            301
14  1f069ds             62
15  1f094m4           2609
16  1f0vr01            418
17  1f0vtuu           2162
18  1f0yurc            757
19  1f5kvh8           3099
20  1f5nlii            154
21  1f5nnr5             25
22  1f5nve4              6
23  1f5o3c8             88
24  1f5oln4             34
25  1f5rjm0            719
26  1f6d8bc            675
27  1f6darv           1044
28  1f6j9hv            256
29  1fgjren           1594
30  1fgmi4r            443
31  1fgmp6b             20
32  1fgmpft            796
33  1fgmuru             58
34  1fgmyhe             36
35  1fgqcnb            673
3

# Tìm lại những Post bị miss

In [ ]:
# 🔹 Tìm các post chưa cào và chạy lại
async def retry_missing_posts(df_goc, df_ket_qua):
    # 🔥 1. Xác định các post_id chưa cào
    post_ids_goc = set(df_goc["submission_id"].dropna().astype(str))
    post_ids_crawled = set(df_ket_qua["post_id"].dropna().astype(str))
    
    missing_posts = list(post_ids_goc - post_ids_crawled)
    print(f"\n🔄 Tổng số trận chưa cào: {len(missing_posts)}\n")

    if not missing_posts:
        print("✅ Không còn post nào cần cào lại!")
        return df_ket_qua  # Không cần làm gì thêm nếu tất cả đã cào

    # 🔥 2. Cào lại các trận bị thiếu
    reddit = await create_reddit_client()
    semaphore = asyncio.Semaphore(2)  # Giới hạn số request đồng thời

    tasks = {post_id: get_all_comments_async(reddit, post_id, semaphore) for post_id in missing_posts}
    results = await asyncio.gather(*tasks.values())

    await reddit.close()

    # 🔥 3. Tạo dataframe từ dữ liệu mới cào được
    new_comments = [comment for post_comments in results if post_comments for comment in post_comments]
    df_new = pd.DataFrame(new_comments)

    # 🔥 4. Merge dữ liệu mới vào dataframe cũ
    df_final = pd.concat([df_ket_qua, df_new], ignore_index=True)

    # 🔥 5. Sắp xếp theo thời gian
    if "comment_time" in df_final.columns:
        df_final = df_final.sort_values(by="comment_time", ascending=True)

    print(f"\n📊 Tổng số comment sau khi cập nhật: {len(df_final)}")

    return df_final

# 🔹 Load dữ liệu
df_goc = pd.read_csv(r"D:\Desktop\tap tanh hoc code\.vscode\DAP\Reddit_Sentiment_Analysis\dataframe\full_post(finished_matches_order).csv")
df_goc = df_goc[(df_goc['matchday']>=1)&(df_goc['matchday']<=6)]
df_ket_qua = pd.read_csv(r"D:\Desktop\tap tanh hoc code\.vscode\DAP\reddit_comments_updated.csv")


,match_id,utc_date,status,matchday,home_team_x,home_team_tla,home_team_short_name,away_team_x,away_team_tla,away_team_short_name,...,match_key,home_team_y,away_team_y,match_time,post_date,title,url,upvotes,comments,submission_id
0,497410,2024-08-16 19:00:00,FINISHED,1,Manchester United,MUN,Man United,Fulham,FUL,Fulham,...,Manchester United vs Fulham,Manchester United,Fulham,2024-08-16 19:00,2024-08-16 18:55,Match Thread: Manchester United vs Fulham | En...,https://www.reddit.com/r/soccer/comments/1etwv...,175.0,3826.0,1etwv7n
1,497411,2024-08-17 11:30:00,FINISHED,1,Ipswich Town,IPS,Ipswich Town,Liverpool,LIV,Liverpool,...,Ipswich Town vs Liverpool,Ipswich Town,Liverpool,2024-08-17 11:30,2024-08-17 11:18,Match Thread: Ipswich Town vs Liverpool | Engl...,https://www.reddit.com/r/soccer/comments/1eufb...,103.0,2349.0,1eufbfh
2,497412,2024-08-17 14:00:00,FINISHED,1,Arsenal,ARS,Arsenal,Wolverhampton Wanderers,WOL,Wolverhampton,...,Arsenal vs Wolverhampton Wanderers,Arsenal,Wolverhampton Wanderers,2024-08-17 14:00,2024-08-17 13:43,Match Thread: Arsenal vs Wolverhampton Wandere...,https://www.reddit.com/r/soccer/comments/1eui1...,57.0,1375.0,1eui1ym
3,497413,2024-08-17 14:00:00,FINISHED,1,Everton,EVE,Everton,Brighton & Hove Albion,BHA,Brighton Hove,...,Everton vs Brighton & Hove Albion,Everton,Brighton & Hove Albion,2024-08-17 14:00,2024-08-17 13:43,Match Thread: Everton vs Brighton & Hove Albio...,https://www.reddit.com/r/soccer/comments/1eui2...,21.0,196.0,1eui26i
4,497414,2024-08-17 14:00:00,FINISHED,1,Newcastle United,NEW,Newcastle,Southampton,SOU,Southampton,...,Newcastle United vs Southampton,Newcastle United,Southampton,2024-08-17 14:00,2024-08-17 13:45,Match Thread: Newcastle United vs Southampton ...,https://www.reddit.com/r/soccer/comments/1eui3...,29.0,267.0,1eui3pz


In [42]:
print(df_goc.shape)
print(len(set(df_ket_qua['post_id'])))

(60, 25)
57


In [45]:
import pandas as pd

# Load dữ liệu
df_goc = pd.read_csv(r"D:\Desktop\tap tanh hoc code\.vscode\DAP\Reddit_Sentiment_Analysis\dataframe\full_post(finished_matches_order).csv")
df_goc = df_goc[(df_goc['matchday'] >= 1) & (df_goc['matchday'] <= 6)]

df_ket_qua = pd.read_csv(r"D:\Desktop\tap tanh hoc code\.vscode\DAP\reddit_comments_updated.csv")

# Chuyển post_id về dạng string để tránh lỗi khi so sánh
df_goc['submission_id'] = df_goc['submission_id'].astype(str)
df_ket_qua['post_id'] = df_ket_qua['post_id'].astype(str)

# Tạo danh sách các post_id đã có comment
post_ids_goc = set(df_goc["submission_id"].dropna())
post_ids_crawled = set(df_ket_qua["post_id"].dropna())

# 🔥 Tìm post_id chưa được cào comment
missing_posts = post_ids_goc - post_ids_crawled

print(f"🔍 Tổng số trận cần cào lại: {(missing_posts)}")

# 🔹 Lọc các trận chưa được cào từ df_goc
df_missing = df_goc[df_goc["submission_id"].isin(missing_posts)]
df_missing.to_csv("missing_posts.csv", index=False)

print("\n✅ Đã lưu danh sách post cần cào vào 'missing_posts.csv'!")


🔍 Tổng số trận cần cào lại: {'nan'}

✅ Đã lưu danh sách post cần cào vào 'missing_posts.csv'!


In [33]:
async def process_posts_from_csv(csv_file):
    df = pd.read_csv(csv_file)
    df = df[(df["matchday"] >= 1) & (df["matchday"] <= 6)]
    post_ids = df["submission_id"].dropna().astype(str).tolist()
    print(f"🔄 Tổng số trận đấu cần cào: {len(post_ids)}\n")

    reddit = await create_reddit_client()
    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

    tasks = [get_all_comments_async(reddit, post_id, semaphore) for post_id in post_ids]
    results = await asyncio.gather(*tasks)

    await reddit.close()

    all_comments = [comment for post_comments in results for comment in post_comments]
    print(f"\n📊 Tổng số comment lấy được: {len(all_comments)}")
    return all_comments

# 🔹 Chạy script
csv_file = r"D:\Desktop\tap tanh hoc code\.vscode\DAP\missing_posts.csv"
comments = asyncio.run(process_posts_from_csv(csv_file))

# 🔹 Lưu kết quả vào file CSV
df_missing_comments = pd.DataFrame(comments)
df_missing_comments.to_csv("reddit_missing_comments.csv", index=False)

print(f"\n✅ Hoàn thành! Đã lấy {len(comments)} comment và lưu vào reddit_comments.csv.")


🔄 Tổng số trận đấu cần cào: 2



C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23888\2516041929.py:25: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print(f"Remaining requests: {remaining}, Reset time: {datetime.utcfromtimestamp(reset_time).strftime('%Y-%m-%d %H:%M:%S')}")


Remaining requests: 834.0, Reset time: 2025-03-05 06:49:57
📌 [Lần 1] Đang lấy comment của post: 1fhbl7c...


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000002510CAA6CC0>


✅ Hoàn tất post 1fhbl7c, lấy được 4930 comment.
⏳ Chờ 1.00s trước khi tiếp tục...
Remaining requests: 85.0, Reset time: 2025-03-05 06:49:57
📌 [Lần 1] Đang lấy comment của post: 1fmvl6j...
⚠️ Lỗi khi lấy comment của post 1fmvl6j (Lần 1/3): received 429 HTTP response
🔁 Thử lại sau 1s...
📌 [Lần 2] Đang lấy comment của post: 1fmvl6j...
✅ Hoàn tất post 1fmvl6j, lấy được 11120 comment.
⏳ Chờ 1.00s trước khi tiếp tục...

📊 Tổng số comment lấy được: 16050

✅ Hoàn thành! Đã lấy 16050 comment và lưu vào reddit_comments.csv.


In [34]:

# 🔹 Gộp dữ liệu mới vào dataframe cũ
df_final = pd.concat([df_ket_qua, df_missing_comments], ignore_index=True)

# 🔹 Sort theo thời gian nếu có cột `comment_time`
if "comment_time" in df_final.columns:
    df_final = df_final.sort_values(by="comment_time", ascending=True)

# 🔹 Lưu file sau khi cập nhật
df_final.to_csv("reddit_comments_updated.csv", index=False)
print("\n✅ Đã lưu dữ liệu cập nhật vào 'reddit_comments_updated.csv'!")



✅ Đã lưu dữ liệu cập nhật vào 'reddit_comments_updated.csv'!
